**Insper**  
**Análise de Textos de Fontes Desestruturadas e WEB**

# **IMDB: <br/><SUBTÍTULO>**

**Beatriz de Jesus**  
**Luciano Felix**  
**Rodrigo Villela**


## **1. Introdução**

> Lorem ipsum dolor sit amet rebum blandit sit diam molestie ea. Erat elitr dolor magna erat velit sadipscing est. Sadipscing accusam elitr vel sea dolore at sed sed voluptua vero at no ut.

Tation dolor suscipit lorem ipsum duo gubergren accusam lorem feugiat diam voluptua gubergren erat dolore diam qui. Eum voluptua consequat ipsum ipsum magna sadipscing sed sit aliquyam diam lorem accumsan consetetur justo aliquyam. Accusam molestie duo nonumy no clita eirmod elitr. Quis volutpat magna lorem vel molestie takimata vero justo eos ipsum tempor ipsum at consetetur exerci eirmod lorem aliquyam. Takimata sit ut aliquyam. Sed tempor consequat kasd molestie ea amet gubergren stet et amet amet stet invidunt euismod in dolore vero. Duis invidunt dolores vel. Vero et feugiat sea eirmod accusam. Sadipscing molestie voluptua rebum magna in dolore sit elitr ea in commodo. Lorem ad at minim et stet at velit clita ea eirmod clita et amet labore. Dolor at eos sit vero dolor sed tempor. Nonumy invidunt eirmod. Aliquam amet veniam et sadipscing takimata sit amet justo nisl at amet at aliquyam clita eirmod. In hendrerit autem blandit nonumy. Consequat labore rebum ipsum tempor eirmod placerat nonumy et te et dolore. Autem nonumy magna nonumy eirmod erat labore et ipsum et autem accusam consetetur. Duo erat minim.


In [1]:
from dataclasses import dataclass
from typing import List, Dict
from urllib.parse import urlparse
from pathlib import PurePath

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd


BASE_REQUESTS_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}


In [99]:
def get_top_250_ids(headers: Dict[str, str] = BASE_REQUESTS_HEADERS) -> List[str]:
    url = "https://www.imdb.com/chart/top"
    response = requests.get(url, headers=headers)
    soup = bs(response.text)

    return [
        PurePath(url.path).name
        for node in soup.select('table > tbody > tr > td.titleColumn > a')
        for url in urlparse(node.attrs["href"])
    ]

top_250_ids = get_top_250_ids()
top_250_ids[:5]

['tt0111161',
 'tt0068646',
 'tt0468569',
 'tt0071562',
 'tt0050083',
 'tt0108052',
 'tt0167260',
 'tt0110912',
 'tt0120737',
 'tt0060196',
 'tt0109830',
 'tt0137523',
 'tt0167261',
 'tt1375666',
 'tt0080684',
 'tt0133093',
 'tt0099685',
 'tt0073486',
 'tt0114369',
 'tt0038650',
 'tt0047478',
 'tt0102926',
 'tt0120815',
 'tt0317248',
 'tt0816692',
 'tt0118799',
 'tt0120689',
 'tt0076759',
 'tt0103064',
 'tt0088763',
 'tt0245429',
 'tt0253474',
 'tt0054215',
 'tt6751668',
 'tt0110413',
 'tt0172495',
 'tt0110357',
 'tt0120586',
 'tt0407887',
 'tt2582802',
 'tt0482571',
 'tt0114814',
 'tt0034583',
 'tt0095327',
 'tt0056058',
 'tt1675434',
 'tt0027977',
 'tt0064116',
 'tt0095765',
 'tt0047396',
 'tt0078748',
 'tt0021749',
 'tt0078788',
 'tt0209144',
 'tt1853728',
 'tt0082971',
 'tt0910970',
 'tt0405094',
 'tt0043014',
 'tt0050825',
 'tt0081505',
 'tt0032553',
 'tt4154756',
 'tt0051201',
 'tt0090605',
 'tt4633694',
 'tt0169547',
 'tt0057012',
 'tt1345836',
 'tt0361748',
 'tt0364569',
 'tt00

In [100]:
@dataclass
class Title:
    title: str
    score: float
    plot: str

def fetch_title(title_id: str, headers: Dict[str, str] = BASE_REQUESTS_HEADERS) -> Title:
    url = f"https://www.imdb.com/title/{title_id}/"
    response = requests.get(url, headers=headers)
    soup = bs(response.text)

    title_node = soup.find("h1")
    score_node = soup.select_one('[data-testid="hero-rating-bar__aggregate-rating__score"]').next_element
    plot_node = soup.select_one('[data-testid="plot-xl"]')

    return Title(
        title = title_node.text,
        score = float(score_node.text),
        plot = plot_node.text
    )


In [102]:
titles = []
for index, title_id in enumerate(top_250_ids, 1):
    print(f"\r{index} of {len(top_250_ids)}", end="")

    titles.append(fetch_title(title_id) )

print(f"\rComplete!")

df = pd.DataFrame(titles)
df.head(5)

Complete!


[Title(title='Um Sonho de Liberdade', score=9.3, plot='Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.'),
 Title(title='O Poderoso Chefão', score=9.2, plot='Don Vito Corleone, head of a mafia family, decides to hand over his empire to his youngest son Michael. However, his decision unintentionally puts the lives of his loved ones in grave danger.'),
 Title(title='Batman: O Cavaleiro das Trevas', score=9.0, plot='When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.'),
 Title(title='O Poderoso Chefão II', score=9.0, plot='The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.'),
 Title(title='12 Homens e uma Sentença', score=9.0, plot='The jury in a New Y

In [105]:
df.to_csv("../data/imdb_top_250.csv")